# 벡터 DB : Chroma vs Pinecone
- Chroma : 인메모리 vector DB, 로컬메모리 vector DB
- Pinecone : 클라우드 vector DB
    (Pinecone console에 api key 생성 -> .env (PINECONE_API_KEY등록)

# 0. 패키지 설치

In [ ]:
%pip install -q pinecone-client langchain-pinecone

# 1. Knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/with_markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)

225

In [3]:
# embedding : upstage embedding-query
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/upstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
    # model="embedding-query"
)

In [5]:
index_name = "tax-index-markdown"

In [ ]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()


database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name
)


c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: total: 15.8 s
Wall time: 1min 11s


In [ ]:
# 업로드한 벡터DB 가져올 때
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore(
    embedding=embedding, # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

# 2. 답변 생성 전 retreival 생성

In [6]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrievetr = database.as_retriever(search_kwargs={"k": 3})

# 3. 제공되는 prompt를 활용하여 답변 생성

In [7]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [8]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [10]:
query = "연봉 5천만원인 거주자의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 약 1,000만원에서 1,200만원 정도입니다. 이는 과세표준이 5,000만원 기준 세율(624만원 + 24%)을 적용하기 때문입니다. 정확한 금액은 기본공제, 연금공제, 자녀공제 등 여러 공제 항목에 따라 달라질 수 있습니다.'}

In [18]:
# 사람을 나타내는 표현 -> 거주자로 변경
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
dictionary = ['사람을 나타내는 표현 -> 거주자']
prompt = ChatPromptTemplate.from_template(f""" 사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의
                                          질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
                                          그런 경우에는 질문만 리턴해 주세요.
                                          사전 : {dictionary}
                                          사용자 질문 : {{query}}""")

In [20]:
dictionary_chain = prompt | llm | StrOutputParser()

In [21]:
dictionary_chain.invoke({"query":query})

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [22]:
new_chain = {'query':dictionary_chain} | qa_chain

In [24]:
new_chain.invoke({"query":query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 약 624만원이 됩니다. 이는 5,000만원 과세표준에 대한 24% 세율을 적용한 금액입니다. 정확한 계산을 위해 근로소득 공제 및 기타 공제액을 고려해야 하지만, 이 금액이 가장 근접한 예상 세금입니다.'}